In [7]:
import requests
import pandas as pd
import numpy as np
import csv
import io
import datetime
from sqlalchemy import create_engine
import pyodbc
import urllib.request
import json
import pyodbc
from sqlalchemy import create_engine
import sqlalchemy
from pandas.io.json import json_normalize
import time
import dateparser
from googletrans import Translator
import timeit
import codecs

replace thai words with english

In [114]:
#### de-duplicate rows
#      - drop contract column as it contains list
#      - de-dup rows
#### combine all of the file
#### translate thai to english


#### Load in pickle data
df_contract_2558 = pd.read_pickle("2558_contract.pkl")
df_project_2558 = pd.read_pickle("2558_project.pkl")

df_contract_2559 = pd.read_pickle("2559_contract.pkl")
df_project_2559 = pd.read_pickle("2559_project.pkl")

df_contract_2560 = pd.read_pickle("2560_contract.pkl")
df_project_2560 = pd.read_pickle("2560_project.pkl")

df_contract_2561 = pd.read_pickle("2561_contract.pkl")
df_project_2561 = pd.read_pickle("2561_project.pkl")

df_contract_2562 = pd.read_pickle("2562_contract.pkl")
df_project_2562 = pd.read_pickle("2562_project.pkl")

In [115]:
#combine project/contract data into a single table
project_frames = [df_project_2558, df_project_2559, df_project_2560, df_project_2561, df_project_2562]
contract_frames = [df_contract_2558, df_contract_2559, df_contract_2560, df_contract_2561, df_contract_2562]

df_project = pd.concat(project_frames)
df_contract = pd.concat(contract_frames)

In [116]:
#Data cleaning

df_project.drop(['contract'], axis = 1, inplace = True)
df_project.drop_duplicates(inplace = True)

df_contract.drop_duplicates(inplace = True)

#df_project.to_pickle("contract_thai_consolidated.pkl")
#df_contract.to_pickle("project_thai_consolidated.pkl")

In [ ]:
#get unique data for each column's date
unique_contract_date = df_contract.contract_date.unique()
unique_contract_finish_date = df_contract.contract_finish_date.unique()
unique_announce_date = df_project.announce_date.unique()
unique_transaction_date = df_project.transaction_date.unique()



#append the date column together

unique_all_date = []
unique_all_date = np.append(unique_all_date,unique_contract_date)
unique_all_date = np.append(unique_all_date,unique_contract_finish_date)
unique_all_date = np.append(unique_all_date,unique_announce_date)
unique_all_date = np.append(unique_all_date,unique_transaction_date)

#get overall unique date
unique_all_date = np.unique(unique_all_date)

#save to a file
np.save("unique_date.npy", unique_all_date)




##### save list of unique_dept_name
unique_dept_name = df_project.dept_name.unique()
np.save("unique_dept_name.npy", unique_dept_name)

In [139]:
unique_project_name = df_project.project_name.unique()
np.save("unique_project_name.npy", unique_project_name)

unique_dept_sub_name = df_project.dept_sub_name.unique()
np.save("unique_dept_sub_name.npy", unique_dept_sub_name)

unique_winner = df_contract.winner.unique()
np.save("unique_winner.npy", unique_winner)

unique_sub_district = df_project.subdistrict.unique()
np.save("unique_sub_district.npy", unique_sub_district)

In [2]:
df_contract = pd.read_pickle("contract_thai_consolidated.pkl")
df_project = pd.read_pickle("project_thai_consolidated.pkl")

In [3]:
#Data transformation

'''
['project_id',                  # 400,988 ID          ------------------------------------------------------------
'project_name',                 # 375,355 project name      ***** too many characters can't translate *****
'project_type_name',            # 8 project type      ------------------------------------------------------------
'dept_name',                    # 18,322 department  -------------------------------------------------------------
'dept_sub_name',                # 27,425 sub department     ***** too many characters can't translate *****
'purchase_method_group_name'    # 12 methods          ------------------------------------------------------------
'announce_date',                # thai date time, need to convert to english   -----------------------------------
'project_money',                # Thai bath but in string with comma, need to transform -------------------------     
'price_build',                  # Thai bath but in string with comma, need to transform -------------------------         
'sum_price_agree',              # Thai bath but in string with comma, need to transform -------------------------
'budget_year',                  # Year in thai, need to convert to English      --------------
'transaction_date'              # thai date time, need to convert to english     ---------------------------
, 'province',                   # thai province, 78                  -------------------------------
'district',                     # 923 distinct                       -------------------------------
'subdistrict',                  # 5,004 distict                      ***** still pending *****
'project_status',               # project_status                     -------------------------------
'geom',                         # geom 


'winner_tin'                   # winner ID                          ----------------
'winner'                       # company's name in thai/english     ------------------
'contract_no'                  # number                             -----------------
'contract_date'                # thai date time, need to convert to english     -------------------
'contract_finish_date'         # thai date time, need to convert to english     ---------------
'price_agree'                  # Thai bath but in string with comma, need to transform    -------------
'status'                       # thai word                         ------------------
'project_id'                   # unique ID                          ----------------

'''


"\n['project_id',                  # 400,988 ID\n'project_name',                 # 375,355 project name \n'project_type_name',            # 8 project type      --------------\n'dept_name',                    # 18,322 department\n'dept_sub_name',                # 27,425 sub department\n'purchase_method_group_name'    # 12 methods          -------------\n'announce_date',                # thai date time, need to convert to english\n'project_money',                # Thai bath but in string with comma, need to transform\n'price_build',                  # Thai bath but in string with comma, need to transform\n'sum_price_agree',              # Thai bath but in string with comma, need to transform\n'budget_year',                  # Year in thai, need to convert to English\n'transaction_date'              # thai date time, need to convert to english\n, 'province',                   # thai province, 78    ------------\n'district',                     # 923 distinct         ------------\n'subdist

In [246]:
project_type_name_th = ['จ้างก่อสร้าง', 'จ้างทำของ/จ้างเหมาบริการ', 'ซื้อ', 'เช่า',
       'จ้างที่ปรึกษา', 'จ้างควบคุมงาน', 'จ้างออกแบบ',
       'จ้างออกแบบและควบคุมงาน']

project_type_name_eng = ['Hire for construction', 'Outsourcing / Contracting services', 'buy', 'rent',
        'Hire consultants', 'Hire to supervise', 'Hire to design',
        'Hire to design and supervise']

purchase_method_group_name_th = ['ประกวดราคา', 'ประกวดราคาด้วยวิธีการทางอิเล็กทรอนิกส์', 'พิเศษ',
       'จ้างที่ปรึกษา', 'กรณีพิเศษ',
       'ประกวดราคาอิเล็กทรอนิกส์ (e-bidding)', 'สอบราคา', 'ตกลงราคา',
       'ตลาดอิเล็กทรอนิกส์ (e-market)', 'จ้างออกแบบ', 'คัดเลือก',
       'เฉพาะเจาะจง']

purchase_method_group_name_eng = ['Bidding', 'E-bidding', 'Special',
        'Hire consultants', 'Special cases',
        'E-bidding','Price examination','Agreed price',
        'E-market', 'Hire to design', 'selection',
        'Specific']

province_th = ['นนทบุรี', 'กรุงเทพมหานคร', 'อุตรดิตถ์', 'เชียงราย', 'ชลบุรี',
       'พระนครศรีอยุธยา', 'สมุทรปราการ', 'ปทุมธานี', 'ลำปาง', 'นครนายก',
       'เชียงใหม่', 'ขอนแก่น', 'พะเยา', 'ปัตตานี', 'ลพบุรี',
       'อุบลราชธานี', 'ยะลา', 'นครราชสีมา', 'อุดรธานี', 'สมุทรสาคร',
       'บึงกาฬ', 'จันทบุรี', 'ฉะเชิงเทรา', 'นครศรีธรรมราช', 'สงขลา',
       'นครปฐม', 'มหาสารคาม', 'กาญจนบุรี', 'แพร่', 'พิจิตร', 'ภูเก็ต',
       'พิษณุโลก', 'กาฬสินธุ์', 'กำแพงเพชร', 'สุราษฎร์ธานี', 'ระยอง',
       'ศรีสะเกษ', 'เพชรบุรี', 'เลย', 'ชัยภูมิ', 'สกลนคร', 'นครพนม',
       'บุรีรัมย์', 'สระบุรี', 'ปราจีนบุรี', 'สุรินทร์', 'นครสวรรค์',
       'เพชรบูรณ์', 'นราธิวาส', 'ประจวบคีรีขันธ์', 'ลำพูน', 'ยโสธร',
       'สุโขทัย', 'สตูล', 'พังงา', 'ชัยนาท', 'กระบี่', 'พัทลุง', 'ชุมพร',
       'ตาก', 'ตรัง', 'อำนาจเจริญ', 'ราชบุรี', 'ร้อยเอ็ด', 'หนองคาย',
       'มุกดาหาร', 'สิงห์บุรี', 'สมุทรสงคราม', 'สุพรรณบุรี', 'อ่างทอง',
       'ตราด', 'ระนอง', 'น่าน', 'อุทัยธานี', 'สระแก้ว', 'หนองบัวลำภู',
       'แม่ฮ่องสอน', '-']

province_eng = ['Nonthaburi', 'Bangkok', 'Uttaradit', 'Chiang Rai', 'Chon Buri',
        'Phra Nakhon Si Ayutthaya', 'Samut Prakan', 'Pathum Thani', 'Lampang', 'Nakhon Nayok',
        'Chiang Mai', 'Khon Kaen', 'Phayao', 'Pattani', 'Lopburi',
        'Ubon Ratchathani', 'Yala', 'Nakhon Ratchasima', 'Udon Thani', 'Samut Sakhon',
        'Bueng Kan', 'Chanthaburi', 'Chachoengsao', 'Nakhon Si Thammarat', 'Songkhla',
        'Nakhon Pathom', 'Maha Sarakham', 'Kanchanaburi', 'Phrae', 'Phichit', 'Phuket',
        'Phitsanulok', 'Kalasin', 'Kamphaeng Phet', 'Surat Thani', 'Rayong',
        'Sisaket', 'Phetchaburi', 'Loei', 'Chaiyaphum', 'Sakon Nakhon', 'Nakhon Phanom',
        'Buriram', 'Saraburi', 'Prachin Buri', 'Surin', 'Nakhon Sawan',
        'Phetchabun', 'Narathiwat', 'Prachuap Khiri Khan', 'Lamphun', 'Yasothon',
        'Sukhothai', 'Satun', 'Phang Nga', 'Chai Nat', 'Krabi', 'Phatthalung', 'Chumphon',
        'Tak', 'Trang', 'Amnat Charoen', 'Ratchaburi', 'Roi Et', 'Nong Khai',
        'Mukdahan', 'Sing Buri', 'Samut Songkhram', 'Suphan Buri', 'Ang Thong',
        'Trat', 'Ranong', 'Nan', 'Uthai Thani', 'Sa Kaeo', 'Nong Bua Lamphu',
        'Mae Hong Son', '-']

project_status_th = ['ระหว่างดำเนินการ', 'สิ้นสุดสัญญา']
project_status_eng = ['In progress', 'End of contract']



#### file in dictionary format to replace thai to english words
with codecs.open('district_th.txt', encoding='utf-8') as f:
    lines = [line for line in f.readlines()]
    
district_th = [line.replace('\r\n','') for line in lines]

with codecs.open('district_eng.txt', encoding='utf-8') as f:
    lines = [line for line in f.readlines()]
    
district_eng = [line.replace('\r\n','') for line in lines]


with open("unique_date_dict_cleaned.pkl", 'rb') as f:
    unique_date_dict = pickle.load(f)

    
with open("unique_winner_dict.pkl", 'rb') as f:
    unique_winner_dict = pickle.load(f)
    
    
with open("unique_dept_name_dict.pkl", 'rb') as f:
    unique_dept_name_dict = pickle.load(f)

In [10]:
df_project['project_type_name_eng'] = df_project['project_type_name'].replace(project_type_name_th,project_type_name_eng)
df_project['purchase_method_group_name_eng'] = df_project['purchase_method_group_name'].replace(purchase_method_group_name_th,purchase_method_group_name_eng)
df_project['province_eng'] = df_project['province'].replace(province_th,province_eng)
df_project['project_status_eng'] = df_project['project_status'].replace(project_status_th,project_status_eng)
df_project['district_eng'] = df_project['district'].replace(district_th,district_eng)



In [254]:
# replace with english date
df_project['announce_date_eng'] = df_project['announce_date'].replace(unique_date_dict)
df_project['transaction_date_eng'] = df_project['transaction_date'].replace(unique_date_dict)

df_contract['contract_date_eng'] = df_contract['contract_date'].replace(unique_date_dict)
df_contract['contract_finish_date_eng'] = df_contract['contract_finish_date'].replace(unique_date_dict)

df_contract['status_eng'] = df_contract['status'].replace(project_status_th,project_status_eng)


In [247]:
df_project['project_money_cleaned'] = df_project['project_money'].replace(',', '', regex=True)
df_project['price_build_cleaned'] = df_project['price_build'].replace(',', '', regex=True)

df_project['sum_price_agree_cleaned'] = df_project['sum_price_agree'].replace(',', '', regex=True)
df_project['project_money_cleaned'] = df_project['project_money'].replace(',', '', regex=True)

df_contract['winner_eng'] = df_contract['winner'].replace(unique_winner_dict)

df_project['dept_name_eng'] = df_project['dept_name'].replace(unique_dept_name_dict)
df_project['budget_year_cleaned'] = df_project['budget_year'].astype(int)-543

In [248]:
df_contract['price_agree_eng'] = df_contract['price_agree'].replace(',', '', regex=True)

In [249]:
df_project.to_csv('project_all_consolidated.csv', index = False)
df_contract.to_csv('contract_all_consolidated.csv', index = False)

In [252]:
project_eng_col = ['project_type_name_eng','purchase_method_group_name_eng'
                   ,'province_eng','project_status_eng','district_eng','announce_date_eng','transaction_date_eng',
                  'project_money_cleaned','price_build_cleaned','sum_price_agree_cleaned',
                  'dept_name_eng','budget_year_cleaned','project_id','project_name','geom']

contract_eng_col = ['winner_tin','contract_date_eng','contract_finish_date_eng','status_eng'
                    ,'winner_eng','contract_no','project_id','price_agree_eng']

In [255]:
df_project_eng = df_project[project_eng_col]
df_contract_eng = df_contract[contract_eng_col]

In [256]:
df_project_eng.to_csv('project_eng_consolidated.csv', index = False)
df_contract_eng.to_csv('contract_eng_consolidated.csv', index = False)

In [257]:
df_contract = pd.read_csv("contract_eng_consolidated.csv")
df_project = pd.read_csv("project_eng_consolidated.csv")

C:\Users\v-krvibh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [259]:
df_project.head()

,project_type_name_eng,purchase_method_group_name_eng,province_eng,project_status_eng,district_eng,announce_date_eng,transaction_date_eng,project_money_cleaned,price_build_cleaned,sum_price_agree_cleaned,dept_name_eng,budget_year_cleaned,project_id,project_name,geom
0,Hire for construction,Bidding,Nonthaburi,In progress,Bang Kruai,2 Apr 2013,9 March 2015,0,0,70624000000,Electricity Generating Authority of Thailand,2015,56045006726,ประกวดราคาจ้างซื้อและจ้างก่อสร้างโรงไฟฟ้าเพื่อ...,POINT(100.498178770987 13.8068549692447)
1,Outsourcing / Contracting services,Bidding,Nonthaburi,In progress,Bang Kruai,21 Nov 2014,26 Aug 2015,30000000000,26910000000,22871000000,Electricity Generating Authority of Thailand,2015,57115200071,ประกวดราคาจ้างเหมาขุด-ขนดินและถ่าน ที่เหมืองแม...,POINT(100.498178770987 13.8068549692447)
2,Outsourcing / Contracting services,E-bidding,Bangkok,In progress,Din,27 Aug 2014,13 Oct 2014,4455190000,4480010000,4436575000,Bangkok,2015,57085094977,ประกวดราคาจ้างเหมาเอกชนขนมูลฝอยจากศูนย์กำจัดมู...,POINT(100.565063509801 13.7780980259196)
3,buy,Special,Bangkok,In progress,Dusit,NaN,28 Jan 2015,2800000000,2799999972,2725935375,Army,2015,57085194402,ซื้อฮ.ท.ขนาดเบา แบบที่ 2 จำนวน 6 เครื่อง โดยวิ...,POINT(100.519873059321 13.7873113160748)
4,Hire for construction,E-bidding,Bangkok,In progress,Chatuchak,25 Mar 2014,9 Oct 2014,2599999769,2599999769,2588950000,Army,2015,57035180671,ประกวดราคาจ้างก่อสร้างอาคารเฉลิมพระเกียรติ สมเ...,POINT(100.584152190858 13.8453156541146)
